# Ref-A
* GCR = 0.4
* Albedo = 0.2
* Hub Height = 1.5
* Configuration = 1-Up portrait
* Ground surface = Horizontal
***
* A GCR of 0.4 for the Trina Vertex module (length = 2.384m) corresponds to a rowto-row spacing or pitch of 5.96m
* 1-up Portrait: 5 rows, each with 25 modules
* Torque tube diameter = 15 cm. (round)
* Maximum tracker rotation angle = 55 deg
* Backtracking is enabled
* Location: Albuquerque, New Mexico USA (35.05°, -106.54°)

In [1]:
import os
from pathlib import Path

testfolder = 'Scenarios/Ref-A-1'

if not os.path.exists(testfolder):
    os.makedirs(testfolder)
    
print ("Your simulation will be stored in %s" % testfolder)

Your simulation will be stored in Scenarios/Ref-A-1


In [2]:
import bifacial_radiance as br
import bifacialvf as bf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, platform
import csv
from pathlib import Path

In [3]:
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("bifacial_radiance version ", br.__version__)
print("pyplot ", plt.matplotlib.__version__)
print("bifacialvf version ", bf.__version__)

Working on a  Darwin 23.3.0
Python version  3.11.5 (main, Sep 11 2023, 08:19:27) [Clang 14.0.6 ]
Pandas version  2.0.3
bifacial_radiance version  0.4.2+248.gd973b3e.dirty
pyplot  3.7.2
bifacialvf version  0.1.8.1


In [4]:
simulationname = 'Ref-A-1'

# Location Albuquerque, New Mexico, USA
lat = 35.05
lon = -106.54

# Scene Parameters
azimuth=90
tilt=30

# MakeModule Parameters
moduletype='PVmod'
numpanels=1
module_x = 1.303 # m
module_y = 2.384 # m. slope we will measure
sensorsy=2
torquetube_diam = 0.15

zgap = 0.02 # m
xgap = 0.002 # m

# SceneDict Parameters
pitch = 5.96 # m
albedo = 0.2
hub_height = 1.5 # m  
nMods = 25 
nRows = 5

sceneDict = {'tilt':tilt,'pitch':pitch,'albedo': albedo,'hub_height':hub_height,'azimuth':azimuth, 'nMods': nMods, 'nRows': nRows} 

In [5]:
demo = br.RadianceObj(simulationname,path = testfolder)
demo.setGround(albedo)

path = Scenarios/Ref-A-1
Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.


In [6]:
cumulativesky = False
trackerParams = {
    'limit_angle': 55,
    'backtrack': True,
    'gcr': 0.4,
    'azimuth': 180,  # axis angle, N-S = 180
    'cumulativesky': cumulativesky
}

In [7]:
module=demo.makeModule(name=moduletype,x=module_x,y=module_y, xgap=xgap, zgap=zgap)


Module Name: PVmod
Module PVmod updated in module.json
Pre-existing .rad file objects/PVmod.rad will be overwritten



In [8]:
module.addTorquetube(diameter=torquetube_diam, tubetype='Round', material='Metal_Grey', 
                     axisofrotation=True, visible=True, recompile=True)

Module PVmod updated in module.json
Pre-existing .rad file objects/PVmod.rad will be overwritten



In [9]:
weatherfile = str(Path().resolve().parent.parent / 'Instructions' / 'Phase2_meteo_hourly_psm3format.csv')
weatherfile

'/Users/storopov/Documents/GitHub/bifacial_radiance/docs/development/Instructions/Phase2_meteo_hourly_psm3format.csv'

In [10]:
metdata = demo.readWeatherFile(weatherfile, source='sam', starttime='2022-02-01_0000', endtime='2022-03-01_0000')

COLUMN DATAS Index(['Year', 'Month', 'Day', 'Hour', 'Minute', 'GHI', 'DNI', 'DHI',
       'temp_air', 'wind_speed', 'humidity', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17'],
      dtype='object')
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2022
Filtering dates
Saving file EPWs/metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is left-labeled  with a delta of +30 mins. i.e. 12 is 12:30 sunpos.


In [11]:
trackerdict = demo.set1axis(**trackerParams)

In [12]:
demo.gendaylit1axis()

Creating ~266 skyfiles. 
Created 248 skyfiles in /skies/


{'2022-02-01_0730': {'surf_azm': 90.0,
  'surf_tilt': 17.77,
  'theta': -17.77,
  'dni': 0,
  'ghi': 59,
  'dhi': 56,
  'temp_air': 4.0,
  'wind_speed': 1.7,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-02-01_0730.rad'},
 '2022-02-01_0830': {'surf_azm': 90.0,
  'surf_tilt': 55.0,
  'theta': -55.0,
  'dni': 40,
  'ghi': 138,
  'dhi': 126,
  'temp_air': 4.1,
  'wind_speed': 1.5,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-02-01_0830.rad'},
 '2022-02-01_0930': {'surf_azm': 90.0,
  'surf_tilt': 49.11,
  'theta': -49.11,
  'dni': 274,
  'ghi': 316,
  'dhi': 204,
  'temp_air': 6.9,
  'wind_speed': 1.0,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-02-01_0930.rad'},
 '2022-02-01_1030': {'surf_azm': 90.0,
  'surf_tilt': 29.83,
  'theta': -29.83,
  'dni': 4,
  'ghi': 231,
  'dhi': 224,
  'temp_air': 9.0,
  'wind_speed': 2.1,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-02-01_1030.rad'},
 '2022-02-01_1130': {'surf_azm': 90.0,
  'surf_tilt': 7.65,
  'theta': -7.65,
  'dni': 12,
  'ghi': 232,
  'dhi':

In [13]:
trackerdict = demo.makeScene1axis(module=moduletype, sceneDict=sceneDict)
trackerdict = demo.makeOct1axis()


Making ~248 .rad files for gendaylit 1-axis workflow (this takes a minute..)
248 Radfiles created in /objects/

Making 248 octfiles in root directory.
Created 1axis_2022-02-01_0730.oct
Created 1axis_2022-02-01_0830.oct
Created 1axis_2022-02-01_0930.oct
Created 1axis_2022-02-01_1030.oct
Created 1axis_2022-02-01_1130.oct
Created 1axis_2022-02-01_1230.oct
Created 1axis_2022-02-01_1330.oct
Created 1axis_2022-02-01_1430.oct
Created 1axis_2022-02-01_1530.oct
Created 1axis_2022-02-01_1630.oct
Created 1axis_2022-02-02_0830.oct
Created 1axis_2022-02-02_0930.oct
Created 1axis_2022-02-02_1030.oct
Created 1axis_2022-02-02_1130.oct
Created 1axis_2022-02-02_1230.oct
Created 1axis_2022-02-02_1330.oct
Created 1axis_2022-02-02_1430.oct
Created 1axis_2022-02-02_1530.oct
Created 1axis_2022-02-02_1630.oct
Created 1axis_2022-02-03_1230.oct
Created 1axis_2022-02-03_1330.oct
Created 1axis_2022-02-03_1430.oct
Created 1axis_2022-02-03_1530.oct
Created 1axis_2022-02-03_1630.oct
Created 1axis_2022-02-04_1030.oc

Created 1axis_2022-02-26_0830.oct
Created 1axis_2022-02-26_0930.oct
Created 1axis_2022-02-26_1030.oct
Created 1axis_2022-02-26_1130.oct
Created 1axis_2022-02-26_1230.oct
Created 1axis_2022-02-26_1330.oct
Created 1axis_2022-02-26_1430.oct
Created 1axis_2022-02-26_1530.oct
Created 1axis_2022-02-26_1630.oct
Created 1axis_2022-02-27_0730.oct
Created 1axis_2022-02-27_0830.oct


In [14]:
#trackerdict = demo.analysis1axis(sensorsy=2) # Middle Module

In [15]:
trackerdict = demo.analysis1axis(sensorsy=2, modWanted = 1) # South Module

Linescan in process: 1axis_2022-02-01_0730_Row3_Module1_Front
Linescan in process: 1axis_2022-02-01_0730_Row3_Module1_Back
Saved: results/irr_1axis_2022-02-01_0730_Row3_Module1.csv
Index: 2022-02-01_0730. Wm2Front: 56.12705. Wm2Back: 7.90583
Linescan in process: 1axis_2022-02-01_0830_Row3_Module1_Front
Linescan in process: 1axis_2022-02-01_0830_Row3_Module1_Back
Saved: results/irr_1axis_2022-02-01_0830_Row3_Module1.csv
Index: 2022-02-01_0830. Wm2Front: 185.08960000000002. Wm2Back: 32.56741666666667
Linescan in process: 1axis_2022-02-01_0930_Row3_Module1_Front
Linescan in process: 1axis_2022-02-01_0930_Row3_Module1_Back
Saved: results/irr_1axis_2022-02-01_0930_Row3_Module1.csv
Index: 2022-02-01_0930. Wm2Front: 428.8160333333333. Wm2Back: 60.32051333333333
Linescan in process: 1axis_2022-02-01_1030_Row3_Module1_Front
Linescan in process: 1axis_2022-02-01_1030_Row3_Module1_Back
Saved: results/irr_1axis_2022-02-01_1030_Row3_Module1.csv
Index: 2022-02-01_1030. Wm2Front: 212.74375000000003. 

Linescan in process: 1axis_2022-02-05_1030_Row3_Module1_Back
Saved: results/irr_1axis_2022-02-05_1030_Row3_Module1.csv
Index: 2022-02-05_1030. Wm2Front: 754.6955333333333. Wm2Back: 106.95318333333333
Linescan in process: 1axis_2022-02-05_1130_Row3_Module1_Front
Linescan in process: 1axis_2022-02-05_1130_Row3_Module1_Back
Saved: results/irr_1axis_2022-02-05_1130_Row3_Module1.csv
Index: 2022-02-05_1130. Wm2Front: 724.1818000000001. Wm2Back: 126.27818333333332
Linescan in process: 1axis_2022-02-05_1230_Row3_Module1_Front
Linescan in process: 1axis_2022-02-05_1230_Row3_Module1_Back
Saved: results/irr_1axis_2022-02-05_1230_Row3_Module1.csv
Index: 2022-02-05_1230. Wm2Front: 737.0818666666667. Wm2Back: 123.02288333333333
Linescan in process: 1axis_2022-02-05_1330_Row3_Module1_Front
Linescan in process: 1axis_2022-02-05_1330_Row3_Module1_Back
Saved: results/irr_1axis_2022-02-05_1330_Row3_Module1.csv
Index: 2022-02-05_1330. Wm2Front: 790.1771333333334. Wm2Back: 100.46941666666666
Linescan in pr

Linescan in process: 1axis_2022-02-08_1230_Row3_Module1_Back
Saved: results/irr_1axis_2022-02-08_1230_Row3_Module1.csv
Index: 2022-02-08_1230. Wm2Front: 638.5505833333334. Wm2Back: 104.6085
Linescan in process: 1axis_2022-02-08_1330_Row3_Module1_Front
Linescan in process: 1axis_2022-02-08_1330_Row3_Module1_Back
Saved: results/irr_1axis_2022-02-08_1330_Row3_Module1.csv
Index: 2022-02-08_1330. Wm2Front: 575.9240333333333. Wm2Back: 79.50032166666668
Linescan in process: 1axis_2022-02-08_1430_Row3_Module1_Front
Linescan in process: 1axis_2022-02-08_1430_Row3_Module1_Back
Saved: results/irr_1axis_2022-02-08_1430_Row3_Module1.csv
Index: 2022-02-08_1430. Wm2Front: 595.66755. Wm2Back: 71.68243333333334
Linescan in process: 1axis_2022-02-08_1530_Row3_Module1_Front
Linescan in process: 1axis_2022-02-08_1530_Row3_Module1_Back
Saved: results/irr_1axis_2022-02-08_1530_Row3_Module1.csv
Index: 2022-02-08_1530. Wm2Front: 455.796. Wm2Back: 44.586241666666666
Linescan in process: 1axis_2022-02-08_1630_R

Linescan in process: 1axis_2022-02-11_1430_Row3_Module1_Back
Saved: results/irr_1axis_2022-02-11_1430_Row3_Module1.csv
Index: 2022-02-11_1430. Wm2Front: 847.8684666666668. Wm2Back: 70.44624
Linescan in process: 1axis_2022-02-11_1530_Row3_Module1_Front
Linescan in process: 1axis_2022-02-11_1530_Row3_Module1_Back
Saved: results/irr_1axis_2022-02-11_1530_Row3_Module1.csv
Index: 2022-02-11_1530. Wm2Front: 804.24215. Wm2Back: 50.91935666666666
Linescan in process: 1axis_2022-02-11_1630_Row3_Module1_Front
Linescan in process: 1axis_2022-02-11_1630_Row3_Module1_Back
Saved: results/irr_1axis_2022-02-11_1630_Row3_Module1.csv
Index: 2022-02-11_1630. Wm2Front: 291.2271333333333. Wm2Back: 23.369785
Linescan in process: 1axis_2022-02-12_0730_Row3_Module1_Front
Linescan in process: 1axis_2022-02-12_0730_Row3_Module1_Back
Saved: results/irr_1axis_2022-02-12_0730_Row3_Module1.csv
Index: 2022-02-12_0730. Wm2Front: 336.99896666666666. Wm2Back: 26.298305
Linescan in process: 1axis_2022-02-12_0830_Row3_Mo

Linescan in process: 1axis_2022-02-14_1630_Row3_Module1_Back
Saved: results/irr_1axis_2022-02-14_1630_Row3_Module1.csv
Index: 2022-02-14_1630. Wm2Front: 325.60880000000003. Wm2Back: 25.79352
Linescan in process: 1axis_2022-02-15_0730_Row3_Module1_Front
Linescan in process: 1axis_2022-02-15_0730_Row3_Module1_Back
Saved: results/irr_1axis_2022-02-15_0730_Row3_Module1.csv
Index: 2022-02-15_0730. Wm2Front: 290.7880333333333. Wm2Back: 23.175761666666666
Linescan in process: 1axis_2022-02-15_0830_Row3_Module1_Front
Linescan in process: 1axis_2022-02-15_0830_Row3_Module1_Back
Saved: results/irr_1axis_2022-02-15_0830_Row3_Module1.csv
Index: 2022-02-15_0830. Wm2Front: 807.2606499999999. Wm2Back: 55.702965
Linescan in process: 1axis_2022-02-15_0930_Row3_Module1_Front
Linescan in process: 1axis_2022-02-15_0930_Row3_Module1_Back
Saved: results/irr_1axis_2022-02-15_0930_Row3_Module1.csv
Index: 2022-02-15_0930. Wm2Front: 822.3600833333334. Wm2Back: 79.73394333333334
Linescan in process: 1axis_2022-0

Linescan in process: 1axis_2022-02-18_0930_Row3_Module1_Back
Saved: results/irr_1axis_2022-02-18_0930_Row3_Module1.csv
Index: 2022-02-18_0930. Wm2Front: 834.8172333333333. Wm2Back: 81.20844833333334
Linescan in process: 1axis_2022-02-18_1030_Row3_Module1_Front
Linescan in process: 1axis_2022-02-18_1030_Row3_Module1_Back
Saved: results/irr_1axis_2022-02-18_1030_Row3_Module1.csv
Index: 2022-02-18_1030. Wm2Front: 801.2045166666667. Wm2Back: 112.60976666666667
Linescan in process: 1axis_2022-02-18_1130_Row3_Module1_Front
Linescan in process: 1axis_2022-02-18_1130_Row3_Module1_Back
Saved: results/irr_1axis_2022-02-18_1130_Row3_Module1.csv
Index: 2022-02-18_1130. Wm2Front: 774.4594666666667. Wm2Back: 131.09716666666668
Linescan in process: 1axis_2022-02-18_1230_Row3_Module1_Front
Linescan in process: 1axis_2022-02-18_1230_Row3_Module1_Back
Saved: results/irr_1axis_2022-02-18_1230_Row3_Module1.csv
Index: 2022-02-18_1230. Wm2Front: 789.278. Wm2Back: 128.5014
Linescan in process: 1axis_2022-02-

Linescan in process: 1axis_2022-02-21_1130_Row3_Module1_Back
Saved: results/irr_1axis_2022-02-21_1130_Row3_Module1.csv
Index: 2022-02-21_1130. Wm2Front: 618.3357666666666. Wm2Back: 99.42902833333335
Linescan in process: 1axis_2022-02-21_1230_Row3_Module1_Front
Linescan in process: 1axis_2022-02-21_1230_Row3_Module1_Back
Saved: results/irr_1axis_2022-02-21_1230_Row3_Module1.csv
Index: 2022-02-21_1230. Wm2Front: 897.4549666666666. Wm2Back: 143.65306666666666
Linescan in process: 1axis_2022-02-21_1330_Row3_Module1_Front
Linescan in process: 1axis_2022-02-21_1330_Row3_Module1_Back
Saved: results/irr_1axis_2022-02-21_1330_Row3_Module1.csv
Index: 2022-02-21_1330. Wm2Front: 829.0473333333332. Wm2Back: 105.06195
Linescan in process: 1axis_2022-02-21_1430_Row3_Module1_Front
Linescan in process: 1axis_2022-02-21_1430_Row3_Module1_Back
Saved: results/irr_1axis_2022-02-21_1430_Row3_Module1.csv
Index: 2022-02-21_1430. Wm2Front: 618.4512666666667. Wm2Back: 75.995165
Linescan in process: 1axis_2022-0

Linescan in process: 1axis_2022-02-24_1530_Row3_Module1_Back
Saved: results/irr_1axis_2022-02-24_1530_Row3_Module1.csv
Index: 2022-02-24_1530. Wm2Front: 920.7321999999999. Wm2Back: 54.76996166666666
Linescan in process: 1axis_2022-02-24_1630_Row3_Module1_Front
Linescan in process: 1axis_2022-02-24_1630_Row3_Module1_Back
Saved: results/irr_1axis_2022-02-24_1630_Row3_Module1.csv
Index: 2022-02-24_1630. Wm2Front: 423.74789999999996. Wm2Back: 31.497975000000004
Linescan in process: 1axis_2022-02-25_0730_Row3_Module1_Front
Linescan in process: 1axis_2022-02-25_0730_Row3_Module1_Back
Saved: results/irr_1axis_2022-02-25_0730_Row3_Module1.csv
Index: 2022-02-25_0730. Wm2Front: 386.83575. Wm2Back: 27.87140666666667
Linescan in process: 1axis_2022-02-25_0830_Row3_Module1_Front
Linescan in process: 1axis_2022-02-25_0830_Row3_Module1_Back
Saved: results/irr_1axis_2022-02-25_0830_Row3_Module1.csv
Index: 2022-02-25_0830. Wm2Front: 850.2847833333333. Wm2Back: 58.937295000000006
Linescan in process: 1a

In [16]:
#trackerdict = demo.analysis1axis(sensorsy=2, modWanted = 25) # North Module

In [17]:
demo.calculateResults()

Bifaciality factor of module stored is  1
No CECModule data passed; using default for Prism Solar BHC72-400


/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/performance.py:64: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  alpha_sc=float(CECMod.alpha_sc),
/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/performance.py:65: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  a_ref=float(CECMod.a_ref),
/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/performance.py:66: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  I_L_ref=float(CECMod.I_L_ref),
/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/performance.py:67: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(

{'2022-02-01_0730': {'surf_azm': 90.0,
  'surf_tilt': 17.77,
  'theta': -17.77,
  'dni': 0,
  'ghi': 59,
  'dhi': 56,
  'temp_air': 4.0,
  'wind_speed': 1.7,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-02-01_0730.rad',
  'radfile': 'objects/1axis2022-02-01_0730__C_1.16520_rtr_5.96000_tilt_17.77000_25modsx5rows_origin0,0.rad',
  'scene': {'module': {'x': 1.303, 'y': 2.384, 'z': 0.02, 'modulematerial': 'black', 'scenex': 1.305, 'sceney': 2.384, 'scenez': 0.095, 'numpanels': 1, 'bifi': 1, 'text': '! genbox black PVmod 1.303 2.384 0.02 | xform -t -0.6515 -1.192 0.095 -a 1 -t 0 2.384 0\r\n! genrev Metal_Grey tube1 t*1.305 0.075 32 | xform -ry 90 -t -0.6525 0 0', 'modulefile': 'objects/PVmod.rad', 'glass': False, 'offsetfromaxis': 0.095, 'xgap': 0.002, 'ygap': 0.0, 'zgap': 0.02}, 'modulefile': 'objects/PVmod.rad', 'hpc': False, 'name': 'Scene0', 'gcr': 0.39999999999999997, 'text': '!xform -rx 17.77 -t 0 0 1.5 -a 25 -t 1.305 0 0 -a 5 -t 0 5.96 0 -i 1 -t -15.66 -11.92 0 -rz 90.0 -t 0 0 0 objec

In [18]:
demo.CompiledResults.set_index('timestamp', inplace=True)
dataframe = demo.CompiledResults
# Grear_mean and Gfront_mean values to be recorded in the excel

In [19]:
dataframe

,module,row,POA_eff,Grear_mean,Gfront_mean,Module_temp,Pout_raw,Pout_Gfront,BGG,BGE,Mismatch,Pout,Wind Speed,DNI,DHI,GHI
timestamp,,,,,,,,,,,,,,,,
2022-02-01_0730,1,3,64.032880,7.905830,56.127050,5.795130,26.136842,22.818318,14.085597,14.543246,0.000321,26.136758,1.7,0,56,59
2022-02-01_0830,1,3,217.657017,32.567417,185.089600,10.284258,90.871695,77.197805,17.595487,17.712797,0.000676,90.871081,1.5,40,126,138
2022-02-01_0930,1,3,489.136547,60.320513,428.816033,21.272756,199.218129,175.524399,14.066758,13.498824,0.000280,199.217571,1.0,274,204,316
2022-02-01_1030,1,3,247.619827,34.876077,212.743750,15.758691,101.654631,87.297150,16.393467,16.446677,0.000738,101.653880,2.1,4,224,231
2022-02-01_1130,1,3,260.640268,32.296385,228.343883,16.106922,106.997412,93.739366,14.143749,14.143519,0.000051,106.997357,1.7,12,223,232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-26_1430,1,3,1009.667520,78.632553,931.034967,31.569850,394.129188,366.511309,8.445714,7.535342,0.000171,394.128513,5.2,1037,45,670
2022-02-26_1530,1,3,981.326500,55.819483,925.507017,30.880693,384.329935,364.582336,6.031233,5.416499,0.000141,384.329392,5.6,974,42,499
2022-02-26_1630,1,3,471.218263,32.255447,438.962817,19.552879,193.055230,180.151449,7.348105,7.162741,0.000011,193.055209,5.7,845,32,288


In [20]:
# dataframe.to_csv(Path().resolve().parent.parent / 'Results' / 'Ref-A.csv', index=True)

In [21]:
demo.exportScenarioData(trackerdict, savefile = Path().resolve().parent.parent / 'Results' / 'Ref-A-Feb.csv')